In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_data = pd.read_csv("../input/titanic/train.csv")
#train_data['Deck'] = train_data.Cabin.str[:1]
train_data['LastName'] = last= train_data.Name.str.extract('^(.+?),', expand = False)
train_data['ModifiedTicket'] = train_data.Ticket.str.split().apply(lambda x : 0 if x[:][-1] == 'LINE' else x[:][-1])
train_data.ModifiedTicket = train_data.ModifiedTicket.values.astype('int64')
train_data['FamilySize'] = train_data.SibSp + train_data.Parch + 1
train_data['CorrectedFare'] = train_data.Fare / test_data.FamilySize
train_data.head()

In [ ]:
print(train_data['Ticket'].nunique())

In [ ]:
test_data = pd.read_csv("../input/titanic/test.csv")
# test_data['Deck'] = test_data.Cabin.str[:1]
test_data['LastName'] = last= test_data.Name.str.extract('^(.+?),', expand = False)
test_data['ModifiedTicket'] = test_data.Ticket.str.split().apply(lambda x : 0 if x[:][-1] == 'LINE' else x[:][-1])
test_data.ModifiedTicket = test_data.ModifiedTicket.values.astype('int64')
test_data['FamilySize'] = test_data.SibSp + test_data.Parch + 1
test_data['CorrectedFare'] = test_data.Fare / test_data.FamilySize
train_data['CorrectedFare'] = train_data.Fare / test_data.FamilySize
test_data.head()

In [ ]:
print(test_data['LastName'].nunique())

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost.sklearn import XGBClassifier

In [ ]:
X_train = train_data
y_train = train_data["Survived"]
y_test = test_data
X_test = test_data
X_train.drop(['Survived'], axis=1, inplace=True)
model = RandomForestClassifier(n_estimators=5000, max_depth=40, random_state=1)
# model = DecisionTreeClassifier()
"""model = XGBClassifier(learning_rate=0.001,n_estimators=2500,
                                max_depth=4, min_child_weight=0,
                                gamma=0, subsample=0.7,
                                colsample_bytree=0.7,
                                scale_pos_weight=1, seed=27,
                                reg_alpha=0.00006,
                                use_label_encoder=False)"""

numeric_features = ["Pclass", "Age", "FamilySize", "CorrectedFare", "ModifiedTicket"]
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median'))])

categorical_features = ["Sex", "Embarked", "LastName"]
categorical_transformer = Pipeline(steps =[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
        ])

my_pipe = Pipeline(steps=[('preprocessor', preprocessor)])
X_transformed_train = my_pipe.fit_transform(X_train, y_train)
X_transformed_test = my_pipe.transform(X_test)
X_transformed_train, X_transformed_valid, y_train, y_valid = train_test_split(X_transformed_train, y_train, 
                                                                train_size=0.8, test_size=0.2,
                                                                random_state=1)
model.fit(X_transformed_train, y_train)
prediction = model.predict(X_transformed_valid)

In [ ]:
print('MAE:', mean_absolute_error(y_valid, prediction))
print("Accuracy of the model: ",round(model.score(X_transformed_train, y_train) * 100, 2))
# model.fit(X_transformed_test, y_test)
# X_transformed_train_OH = my_pipe.transform(X_transformed_test).toarray()
# print(X_transformed_test.shape)
# print(X_transformed_test)
# print(X_transformed_test[22])

In [ ]:
prediction_test = model.predict(X_transformed_test)

In [ ]:
output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': prediction_test})
output.to_csv('my_submission_v2.9.csv', index=False)
print("Your submission was successfully saved!")